# Hidden Markov Model

Assumption:
- $P(y_{t+1}|y_{t}, Z) = P(y_{t+1}|y_{t})$
- $P(x_{t}|y_{t}, Z) = P(x_{t}|y_{t})$

There are 3 kind of probabilities:
- Transmission $P(y_{t+1}|y_{t})$ : likelihood of moving from one state to another state
- Emission $P(x_t|y_t)$ :likelihood of observations given state
- $\pi$- initial state probability distribution.

Forward Algorithm:

$$ \alpha_{t}(j) = P(x1, x2,..., x_t, y_t)
= P(x_t|y_t)\sum_{y_{t-1}}[P(y_t|y_{t-1})P(x1, x2,..., x_{t-1}, y_{t-1})] $$
$$
= P(x_t|y_t)\sum_{y_{t-1}}[P(y_t|y_{t-1})\alpha_{t-1}(j-1)]
$$

dimension($\alpha_{t}(j)$) = (# hiddens x 1) = dimension($y_t$)

dimension($x_t$) = (#observations x 1)

$$ \alpha_{0}(0) = P(x_0, y_0) = P(x_0|y_0)P(y_0) $$ 

P(x_0|y_0) and P(y_0) are intialized.

$$ \alpha_{1}(1) = P(x_0, x_1, y_1) = P(x_1|y_1) \sum_{y_0}[P(y_1|y_0)\alpha_{0}(0)$$
...

In [8]:
import numpy as np

In [3]:
def HMM(T, E):
    """
    Compute 
    
    """
    pass
def train(observations):
    """
    Find what?
    """
    pass

def forward_initialize(E):
    forward = []
    # Asume no information before (actually we can find it) => P(y_0) = 1/len(E) for all states
    P_y0 = 1/len(E.T)
    forward.append(E*P_y0)
    return forward

def forward(new_seq, T, E):
    """
    Return (o x h) array of likelihood of seq of hidden states )
    """
    obs = int(new_seq[0]) - 1 # {1 2 3} -> {0 1 2} to store in array
    forward = forward_initialize(E[obs]) # emission prob from all hidden states to observation [obs]
    
    for t in range(1, len(new_seq)):
        # T.T : transpose of transmission -> T.T[j] = Transmit (i0,i1,..) -> j
        Sum = [T.T[j].dot(forward[t-1]) for j in range(len(T))]
        obser = int(new_seq[t]) - 1
        alpha = Sum * E[obser] # element-wise product
        forward.append(alpha)
    return np.array(forward)

def likelihood(new_seq, T, E):
    """
    Args: 
        new_seq: (1 x o) observation
        T: (dxd) trainsmission - T_{ij} = P(y_j|y_i)is likelihood 
            that state i move to state j
        E: (d) emission - E_{ij} = P(x_j|y_i) is probability of 
            emitting j-th observation state from i-th hidden state
    Return: (1 x h) array of maximum likelihood of seq of hidden states 
    """
    return np.argmax(forward(new_seq, T, E), axis = 1)

In [23]:
emission = np.array([[0.7, 0, .3,.4], 
                     [0.2, 0.3,.4,.3], 
                     [0.1, 0.7,.3,.3]])

transmission = np.array([[0, 0, 0, 0], 
                         [0.5, 0.8, 0.2, 0], 
                         [0.5, 0.1, 0.7, 0], 
                         [0, 0.1, 0.1, 0]])
observations = ['2','3','3','2','3','2','3','2','2','3','1','3','3','1','1',
        '1','2','1','1','1','3','1','2','1','1','1','2','3','3','2',
        '3','2','2']
# HMM(transmission, emission)
# train(observations)
new_seq = ['1', '2', '3']
likelihood(new_seq, transmission, emission)

array([0, 2, 2])

## Refs
https://github.com/aldengolab/hidden-markov-model